# EDA de **Portabilidad Móvil: Portabilidades Completadas**

Cuaderno preparado para **Visual Studio/VS Code** con Python, con explicaciones en Markdown entre pasos y guardado del CSV 'limpio'.

**Fuente de datos (ruta Windows)**:

`C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Portabilidad\\Portabilidad completadas_con_ID.csv`


## 1. Importaciones y configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (11, 5),
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

## 2. Carga del dataset desde la ruta indicada
Se preserva la codificación **UTF‑8 con BOM** para acentos/eñes en Windows.

In [ ]:
ruta = r'C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Portabilidad\\Portabilidad completadas_con_ID.csv'

# Cargar el archivo
df = pd.read_csv(ruta, encoding='utf-8-sig')

# Copia del crudo
df_raw = df.copy()

print('Shape:', df.shape)
print('Columnas:', list(df.columns))
display(df.head())

## 3. Perfil inicial del dataset
- Tipos de datos y memoria (`df.info()`).
- Valores faltantes y duplicados.
- Estadísticos descriptivos de `Total General`.

In [ ]:
df.info()
print('
Nulos por columna:
', df.isna().sum())
print('Duplicados (filas completas):', df.duplicated().sum())

if 'Total General' in df.columns:
    print('
Estadísticos Total General:')
    print(df['Total General'].describe())

## 4. Validaciones de consistencia y plausibilidad
- Conversión a numérico en campos claves.
- Chequeo de negativos y ceros.
- Construcción de **Fecha** y **Periodo** (mensual).

In [ ]:
num_cols = ['Registro_ID','Año','MesId','Total General','TiempoMes_ID','Trimestre','TiempoTrim_ID']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print('
Negativos por columna:
', (df[num_cols] < 0).sum())
print('Ceros en Total General:', (df['Total General'] == 0).sum() if 'Total General' in df.columns else 'N/A')

# Fecha y Periodo mensual
df['Fecha'] = pd.to_datetime(dict(year=df['Año'], month=df['MesId'], day=1))
df['Periodo'] = pd.PeriodIndex(year=df['Año'], month=df['MesId'], freq='M').astype(str)

## 5. Limpieza y ordenamiento
- Eliminación de duplicados.
- Orden temporal por `Año` y `MesId`.

In [ ]:
df = df.drop_duplicates().sort_values(['Año','MesId']).reset_index(drop=True)
display(df.head())

## 6. Variables derivadas
- `MoM_%` (variación mensual).
- `YoY_%` (variación interanual t-12).
- `MA3` (media móvil de 3 meses).

In [ ]:
df['MoM_%'] = df['Total General'].pct_change() * 100
df['YoY_%'] = df['Total General'].pct_change(12) * 100
df['MA3'] = df['Total General'].rolling(3).mean()

display(df[['Año','MesId','Mes','Total General','MoM_%','YoY_%','MA3']].tail(12))

## 7. Visualizaciones
- Serie temporal de **Portabilidades completadas** y su media móvil (MA3).
- Variación `YoY_%` y `MoM_%`.
- (Opcional) Totales por **Trimestre**.

In [ ]:
import matplotlib.ticker as mtick

# 7.1 Portabilidades y media móvil
plt.figure()
sns.lineplot(data=df, x='Periodo', y='Total General', marker='o', label='Total', color='#2E86C1')
sns.lineplot(data=df.dropna(subset=['MA3']), x='Periodo', y='MA3', marker='o', label='Media móvil (3)', color='#7D3C98')
plt.xticks(rotation=45, ha='right')
plt.title('Portabilidades completadas por mes (con MA3)')
plt.xlabel('Periodo'); plt.ylabel('Portabilidades'); plt.legend(); plt.tight_layout(); plt.show()

# 7.2 Variación YoY y MoM
plt.figure()
sns.lineplot(data=df, x='Periodo', y='YoY_%', marker='o', label='YoY %', color='#1ABC9C')
sns.lineplot(data=df, x='Periodo', y='MoM_%', marker='o', label='MoM %', color='#E67E22')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.xticks(rotation=45, ha='right')
plt.title('Variación de Portabilidades (YoY y MoM)')
plt.xlabel('Periodo'); plt.ylabel('%'); plt.legend(); plt.tight_layout(); plt.show()

# 7.3 Totales por Trimestre
df_trim = df.groupby(['Año','Trimestre'], as_index=False)['Total General'].sum()
df_trim['PeriodoTrim'] = pd.PeriodIndex(year=df_trim['Año'], quarter=df_trim['Trimestre'], freq='Q').astype(str)
plt.figure()
sns.barplot(data=df_trim, x='PeriodoTrim', y='Total General', color='#3498DB')
plt.xticks(rotation=45, ha='right')
plt.title('Portabilidades completadas por trimestre')
plt.xlabel('Trimestre'); plt.ylabel('Portabilidades'); plt.tight_layout(); plt.show()

## 8. Guardado del archivo **limpio**
Se guarda el CSV en la **misma ruta** con la **leyenda `_limpio`** agregada al nombre, codificación `utf-8-sig`.

In [ ]:
ruta_path = Path(ruta)
ruta_limpia = ruta_path.with_name(ruta_path.stem + '_limpio' + ruta_path.suffix)
df.to_csv(ruta_limpia, index=False, encoding='utf-8-sig')
print('Guardado en:', ruta_limpia)